ModuleNotFoundError: No module named 'vqgan_with_pos_trial_v2'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
"""
In GroupNorm, channels are divided into groups, and the mean and 
variance are calculated within each group across all instances in a 
batch. This makes GroupNorm's performance less dependent on the batch 
size, and it can maintain its effectiveness even with a batch size of 1. 
This can be particularly useful in applications where memory constraints 
limit the maximum batch size, such as high-resolution image processing.

"""

class GroupNorm(nn.Module):
    def __init__(self, channels):
        super(GroupNorm, self).__init__()
        
        # num_grapups should be divisble by in_channels
        # for example 32 is divisible by 128.
        # 32 is divisible by 256
        # 32 is divisibble by 5122
        self.gn = nn.GroupNorm(num_groups=32, num_channels=channels, eps=1e-6, affine=True)

    def forward(self, x):
        return self.gn(x)


class Swish(nn.Module):
    
    def forward(self, x):
        
        return x * torch.sigmoid(x)


class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        
        super(ResidualBlock, self).__init__()
        
        self.in_channels = in_channels
        
        self.out_channels = out_channels
        
        self.block = nn.Sequential(
            GroupNorm(in_channels),
            Swish(),
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            GroupNorm(out_channels),
            Swish(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1)  # this layer is used to preserve depth which is used in return x + self.block(x)
        )

        if in_channels != out_channels:
            self.channel_up = nn.Conv2d(in_channels, out_channels, 1, 1, 0)

    def forward(self, x):
        if self.in_channels != self.out_channels:
            return self.channel_up(x) + self.block(x)
        else:
            return x + self.block(x)


class UpSampleBlock(nn.Module):
    def __init__(self, channels):
        super(UpSampleBlock, self).__init__()
        self.conv = nn.Conv2d(channels, channels, 3, 1, 1)

    def forward(self, x):
        x = F.interpolate(x, scale_factor=2.0)
        return self.conv(x)

class PositionalEncoding2D(nn.Module):
    def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        pe = torch.zeros(max_len, max_len, d_model)
        position_i = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        position_j = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2.) * -(math.log(10000.0) / d_model))

        pe[:, :, 0::2] = torch.sin(position_i * div_term)
        pe[:, :, 1::2] = torch.cos(position_j * div_term)

        pe = pe.permute(2, 0, 1).unsqueeze(0)  # make shape [1, C, H, W]

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :, :x.size(2), :x.size(3)]
        return self.dropout(x)
    
class DownSampleBlock(nn.Module):
    def __init__(self, channels):
        super(DownSampleBlock, self).__init__()
        self.conv = nn.Conv2d(channels, channels, 3, 2, 0)

    def forward(self, x):
        pad = (0, 1, 0, 1)
        x = F.pad(x, pad, mode="constant", value=0)
        return self.conv(x)

# NonLocal NN: https://arxiv.org/pdf/1711.07971.pdf
class NonLocalBlock(nn.Module):
    def __init__(self, channels):
        super(NonLocalBlock, self).__init__()
        self.in_channels = channels

        self.gn = GroupNorm(channels)
        self.q = nn.Conv2d(channels, channels, 1, 1, 0)
        self.k = nn.Conv2d(channels, channels, 1, 1, 0)
        self.v = nn.Conv2d(channels, channels, 1, 1, 0)
        self.proj_out = nn.Conv2d(channels, channels, 1, 1, 0)

    def forward(self, x):
        h_ = self.gn(x)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        b, c, h, w = q.shape

        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)
        k = k.reshape(b, c, h*w)
        v = v.reshape(b, c, h*w)

        attn = torch.bmm(q, k)
        attn = attn * (int(c)**(-0.5))
        attn = F.softmax(attn, dim=2)
        attn = attn.permute(0, 2, 1)

        A = torch.bmm(v, attn)
        A = A.reshape(b, c, h, w)

        return x + A














In [2]:
import torch.nn as nn
# from helper import ResidualBlock, NonLocalBlock, DownSampleBlock, UpSampleBlock, GroupNorm, Swish

# from PositionalEncoding2dJupyter import PositionalEncoding2D

# from helperjupyter import ResidualBlock, NonLocalBlock, UpSampleBlock, GroupNorm, Swish,PositionalEncoding2D
# Encoder block ==> Page 11/52 (pdf): https://arxiv.org/pdf/2012.09841.pdf

"""_summary_

Step-by-step breakdown of the Encoder module:

Module 1 - Initial Convolution Layer

Layer 1: Conv2d layer (input: image_channels, output: channels[0], kernel_size: 3, stride: 1, padding: 1)
Module 2 - Residual Blocks and Non-local Blocks

For each pair of consecutive channels in the channels list:
Layer 2a: ResidualBlock (input: in_channels, output: out_channels)
Layer 2b: ResidualBlock (input: out_channels, output: out_channels)
If the current resolution is in attn_resolutions, add a NonLocalBlock (input: out_channels)
If not at the last pair of consecutive channels, add a DownSampleBlock (input: out_channels)

Module 3 - Final Layers

Layer 3a: ResidualBlock (input: channels[-1], output: channels[-1])
Layer 3b: NonLocalBlock (input: channels[-1])
Layer 3c: ResidualBlock (input: channels[-1], output: channels[-1])
Layer 3d: GroupNorm (input: channels[-1])
Layer 3e: Swish activation
Layer 3f: Conv2d layer (input: channels[-1], output: latent_dim, kernel_size: 3, stride: 1, padding: 1)
The entire architecture is then built using a nn.Sequential layer containing all the layers from Module 1, Module 2, and Module 3.

channels = [128, 128, 128, 256, 256, 512]
There are 5 instances of Module 2 for the given channel list.

Here's the breakdown:

Module 2.1: Transition from channels[0] (128) to channels[1] (128)
Module 2.2: Transition from channels[1] (128) to channels[2] (128)
Module 2.3: Transition from channels[2] (128) to channels[3] (256)
Module 2.4: Transition from channels[3] (256) to channels[4] (256)
Module 2.5: Transition from channels[4] (256) to channels[5] (512)

The NonLocalBlock is added based on the attn_resolutions list. 
In the given code, attn_resolutions = [16]. The purpose of using the NonLocalBlock is 
to model long-range dependencies within the input feature maps by capturing spatial relationships at specific resolutions.



Returns:
_type_: _description_
"""


# class Encoder(nn.Module):
#     def __init__(self, args):
#         super(Encoder, self).__init__()
#         channels = [128, 128, 128, 256, 256, 512]
#         attn_resolutions = [16]
#         num_res_blocks = 2
#         resolution = 256

#         layers = [nn.Conv2d(args.image_channels, channels[0], 3, 1, 1)]
#         layers.append(PositionalEncoding2D(channels[0], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after first Conv2d
        
#         for i in range(len(channels)-1):
#             in_channels = channels[i]
#             out_channels = channels[i + 1]
#             for j in range(num_res_blocks):
#                 layers.append(ResidualBlock(in_channels, out_channels))
#                 layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#                 in_channels = out_channels
#                 if resolution in attn_resolutions:
#                     layers.append(NonLocalBlock(in_channels))
#                     layers.append(PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#             if i != len(channels)-2: 
#                 layers.append(DownSampleBlock(channels[i+1]))
#                 resolution //= 2
#         layers.append(ResidualBlock(channels[-1], channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#         layers.append(NonLocalBlock(channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#         layers.append(ResidualBlock(channels[-1], channels[-1]))
#         layers.append(PositionalEncoding2D(channels[-1], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#         layers.append(GroupNorm(channels[-1]))
#         layers.append(Swish())
#         layers.append(nn.Conv2d(channels[-1], args.latent_dim, 3, 1, 1))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.model(x)

class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()
        channels = [128, 256, 512]
        attn_resolutions = [16]
        num_res_blocks = 1
        resolution = 256

        layers = [nn.Conv2d(args.image_channels, channels[0], 3, 1, 1)]
        layers.append(PositionalEncoding2D(channels[0], max_len=512, dropout_prob=0.5))  # PositionalEncoding2D at the start
        
        for i in range(len(channels)-1):
            in_channels = channels[i]
            out_channels = channels[i + 1]
            for j in range(num_res_blocks):
                layers.append(ResidualBlock(in_channels, out_channels))
                in_channels = out_channels
                if resolution in attn_resolutions:
                    layers.append(NonLocalBlock(in_channels))
            # if i != len(channels)-1: 
            layers.append(DownSampleBlock(channels[i+1]))
            resolution //= 2
            
            # Adding PositionalEncoding2D in the middle of the encoder
            if i == len(channels) // 2:
                layers.append(PositionalEncoding2D(channels[i+1], max_len=512, dropout_prob=0.5))  
        layers.append(DownSampleBlock(channels[i+1]))
        
        layers.append(ResidualBlock(channels[-1], channels[-1]))
        layers.append(NonLocalBlock(channels[-1]))
        layers.append(DownSampleBlock(channels[i+1]))
        layers.append(ResidualBlock(channels[-1], channels[-1]))
        layers.append(GroupNorm(channels[-1]))
        layers.append(Swish())
        layers.append(nn.Conv2d(channels[-1], args.latent_dim, 3, 1, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [3]:
# import torch
# import torch.nn as nn

# # Your classes here: PositionalEncoding2D, ResidualBlock, NonLocalBlock, DownSampleBlock, GroupNorm, Swish

# class Args:
#     def __init__(self, latent_dim, image_channels, device):
#         self.latent_dim = latent_dim
#         self.image_channels = image_channels
#         self.device = device
# # 
# # class Encoder(nn.Module):
#     # your Encoder code

# # Set up the arguments
# args = Args(latent_dim=256, image_channels=3, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# # Initialize the Encoder
# encoder = Encoder(args)

# # Create a random tensor of size (batch_size, channels, height, width)
# x = torch.randn(5, 3, 256, 256)

# # Pass the tensor through the encoder
# try:
#     y = encoder(x)
#     print("Encoder executed successfully. Output shape:", y.shape)
# except IndexError as e:
#     print("Error occurred during execution of encoder:", str(e))

In [4]:
# class Decoder(nn.Module):
#     def __init__(self, args):
#         super(Decoder, self).__init__()
#         channels = [512, 256, 256, 128, 128]
#         attn_resolutions = [16]
#         num_res_blocks = 3
#         resolution = 16

#         in_channels = channels[0]
#         # first 4 layers of the decoder. 
#         layers = [nn.Conv2d(args.latent_dim, in_channels, 3, 1, 1),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after first Conv2d
#                   ResidualBlock(in_channels, in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after ResidualBlock
#                   NonLocalBlock(in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5),  # PositionalEncoding2D after NonLocalBlock
#                   ResidualBlock(in_channels, in_channels),
#                   PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5)]  # PositionalEncoding2D after ResidualBlock

#         # all other layers. 
#         for i in range(len(channels)):
#             out_channels = channels[i]
#             for j in range(num_res_blocks):
#                 layers.append(ResidualBlock(in_channels, out_channels))
#                 layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after ResidualBlock
#                 in_channels = out_channels
#                 if resolution in attn_resolutions:
#                     layers.append(NonLocalBlock(in_channels))
#                     layers.append(PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D after NonLocalBlock
#             if i != 0:
#                 layers.append(UpSampleBlock(in_channels))
#                 resolution *= 2

#         layers.append(GroupNorm(in_channels))
#         layers.append(Swish())
#         layers.append(nn.Conv2d(in_channels, args.image_channels, 3, 1, 1))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.model(x)
    
from torchinfo import summary

class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()
        channels = [512, 256, 128]
        attn_resolutions = [16]
        num_res_blocks = 1
        resolution = 16

        in_channels = channels[0]
        # first 4 layers of the decoder. 
        layers = [nn.Conv2d(args.latent_dim, in_channels, 3, 1, 1),
                  PositionalEncoding2D(in_channels, max_len=512, dropout_prob=0.5)]  # PositionalEncoding2D after first Conv2d

        # Adding Residual Blocks, NonLocalBlock, and UpSampleBlocks
        for i in range(len(channels)):
            out_channels = channels[i]
            for j in range(num_res_blocks):
                layers.append(ResidualBlock(in_channels, out_channels))
                if i == len(channels) // 2 and j == num_res_blocks // 2:  # Approximately the middle of the network
                    layers.append(PositionalEncoding2D(out_channels, max_len=512, dropout_prob=0.5))  # PositionalEncoding2D
                in_channels = out_channels
                if resolution in attn_resolutions:
                    layers.append(NonLocalBlock(in_channels))
            # if i != 0:
            layers.append(UpSampleBlock(in_channels))
            resolution *= 2

        layers.append(UpSampleBlock(in_channels))
        layers.append(GroupNorm(in_channels))
        layers.append(Swish())
        layers.append(nn.Conv2d(in_channels, args.image_channels, 3, 1, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [5]:
# class Args:
#     def __init__(self):
#         self.latent_dim = 256
#         self.image_channels = 3
#         self.image_size = 256
#         self.batch_size = 5

# args = Args()
# # decoder = Decoder(args)

# # # Create a random tensor with shape (batch_size, latent_dim, height, width)
# # x = torch.randn(1, args.latent_dim, 16, 16)
# # output = decoder(x)

# # print(output.shape)  # Should print torch.Size([1

# # if __name__ == '__main__':
    
# #     parser = argparse.ArgumentParser(description="VQGAN")
# #     parser.add_argument('--latent-dim', type=int, default=256, help='Latent dimension n_z (default: 256)')
# #     parser.add_argument('--image-size', type=int, default=256, help='Image height and width (default: 256)')
# #     parser.add_argument('--num-codebook-vectors', type=int, default=1024, help='Number of codebook vectors (default: 256)')
# #     parser.add_argument('--beta', type=float, default=0.25, help='Commitment loss scalar (default: 0.25)')
# #     parser.add_argument('--image-channels', type=int, default=3, help='Number of channels of images (default: 3)')
# #     parser.add_argument('--dataset-path', type=str, default='/data', help='Path to data (default: /data)')
# #     parser.add_argument('--device', type=str, default="cuda", help='Which device the training is on')
# #     parser.add_argument('--batch-size', type=int, default=3, help='Input batch size for training (default: 6)')
# #     parser.add_argument('--epochs', type=int, default=2, help='Number of epochs to train (default: 50)')
# #     parser.add_argument('--learning-rate', type=float, default=2.25e-05, help='Learning rate (default: 0.0002)')
# #     parser.add_argument('--beta1', type=float, default=0.5, help='Adam beta param (default: 0.0)')
# #     parser.add_argument('--beta2', type=float, default=0.9, help='Adam beta param (default: 0.999)')
# #     parser.add_argument('--disc-start', type=int, default=100, help='When to start the discriminator (default: 0)')
# #     parser.add_argument('--disc-factor', type=float, default=1., help='')
# #     parser.add_argument('--rec-loss-factor', type=float, default=1., help='Weighting factor for reconstruction loss.')
# #     parser.add_argument('--perceptual-loss-factor', type=float, default=1., help='Weighting factor for perceptual loss.')
    
# #     parser.add_argument('--checkpoint-path', type=str, default=None, help='Path to the checkpoint to resume training from (default: None)')
# #     parser.add_argument('--checkpoint-vq-opt-path', type=str, default=None, help='Path to the VQ optimizer checkpoint to resume training from (default: None)')
# #     parser.add_argument('--checkpoint-disc-opt-path', type=str, default=None, help='Path to the discriminator optimizer checkpoint to resume training from (default: None)')

# #     args = parser.parse_args()
#     # Create random input tensor
#     # batch_size = 32
#     # image_channels = 3
#     # height = width = 256
# input_tensor = torch.randn(args.batch_size, args.image_channels, args.image_size, args.image_size)

# # # Create an instance of Encoder
# # latent_dim = 256  # define this as per your requirement
# decoder = Decoder(args)

# # # Print an input and output summary of our Transformer Encoder (uncomment for full output)
# random_input_image = (5, 256,16, 16)
# summary(model=decoder,
#         input_size = random_input_image, 
#         col_names=["input_size", "output_size", "num_params", "trainable"])

In [6]:
import torch
import torch.nn as nn
# from encoder_with_positional_encoding_v2 import Encoder
# from decoder_with_positional_encoding_v2 import Decoder
from codebook import Codebook
import math

class VQGAN(nn.Module):
    def __init__(self, args):
        super(VQGAN, self).__init__()
        self.encoder = Encoder(args).to(device=args.device)
        self.decoder = Decoder(args).to(device=args.device)
        self.codebook = Codebook(args).to(device=args.device)       
        
        # original vq-gan has prequantization and post quantization convolution layers.
        self.quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, 1).to(device=args.device)
        self.post_quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, 1).to(device=args.device)


    def forward(self, imgs):
        encoded_images = self.encoder(imgs)
        # print("encoded_images.shape", encoded_images.shape)
        # pre quantisation layer. 
        quant_conv_encoded_images = self.quant_conv(encoded_images)
        
        
        codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)
        
        # post quantization layer.
        post_quant_conv_mapping = self.post_quant_conv(codebook_mapping)
        
        # print("post_quant_conv_mapping.shape", post_quant_conv_mapping.shape)
        decoded_images = self.decoder(post_quant_conv_mapping)
        # print("decoded_imagess.shape", decoded_images.shape)
        
        return decoded_images, codebook_indices, q_loss
    

    # these functions will be used by the transformers separately.. 
    def encode(self, imgs):
        encoded_images = self.encoder(imgs)
        quant_conv_encoded_images = self.quant_conv(encoded_images)
        codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)
        return codebook_mapping, codebook_indices, q_loss

    def decode(self, z):
        post_quant_conv_mapping = self.post_quant_conv(z)
        decoded_images = self.decoder(post_quant_conv_mapping)
        return decoded_images

    # variable lambda is the weighting factor, between vq-vae loss (or perceptual loss)
    # and the GAN loss..  equation 7 in the vq-gan paper
    # G stands for decoder and L stands for the last layer of decoder. 
    def calculate_lambda(self, perceptual_loss, gan_loss):
        
        # last layer and its weight
        last_layer = self.decoder.model[-1]
        last_layer_weight = last_layer.weight
        
        # gradients for both percetpaul and GAN loss. 
        perceptual_loss_grads = torch.autograd.grad(perceptual_loss, last_layer_weight, retain_graph=True)[0]
        gan_loss_grads = torch.autograd.grad(gan_loss, last_layer_weight, retain_graph=True)[0]

        # clipping lambda value between 0 and 10K and then returning lambda. 
        λ = torch.norm(perceptual_loss_grads) / (torch.norm(gan_loss_grads) + 1e-4)
        λ = torch.clamp(λ, 0, 1e4).detach()
        return 0.8 * λ
    
    """
    
    The idea behind starting the discriminator later in the training process is to 
    give the generator (in this case, the VQGAN's encoder-decoder architecture) some 
    time to learn to reconstruct images before introducing the adversarial training 
    aspect. This strategy can help stabilize the training process and improve convergence.

    In the initial phase of training, the generator learns to reconstruct images 
    without the pressure of fooling the discriminator. When the discriminator is 
    introduced later, the generator has already learned a decent representation of
    the data, which helps it better cope with the adversarial training aspect. This can
    lead to better performance and stability during training.
    
    """
    @staticmethod
    def adopt_weight(disc_factor, i, threshold, value=0.):
        if i < threshold:
            disc_factor = value  # disc_factor is the discriminator weight.. 
        return disc_factor

    def load_checkpoint(self, path):
        self.load_state_dict(torch.load(path))










In [ ]:
# python training_vqgan_v2.py --checkpoint-path "C:/Users/luvve/VQGAN/checkpoints/vqgan_epoch_7.pt"
import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import argparse
from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import utils as vutils
from discriminator import Discriminator
from lpips import LPIPS
# from vqgan_with_pos_trial_v2 import VQGAN
from utils import load_data, weights_init
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor
from PIL import Image
import os
import time



class CustomImageDataset(Dataset):
    def __init__(self, main_directory, transform=None):
        self.main_directory = main_directory
        self.transform = transform
        self.image_files = [f for f in os.listdir(main_directory) if os.path.isfile(os.path.join(main_directory, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.main_directory, self.image_files[idx])
        
        # # Skip directories
        if os.path.isdir(img_path):
            return None

    
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image
    
class TrainVQGAN:
    def __init__(self, args):
        self.vqgan = VQGAN(args).to(device=args.device)
        self.train_loader = self.load_data_2(args.train_folder, args.batch_size)
        self.valid_loader = self.load_data_2(args.valid_folder, args.batch_size)

        self.discriminator = Discriminator(args).to(device=args.device)
        self.discriminator.apply(weights_init)
        self.perceptual_loss = LPIPS().eval().to(device=args.device)
        self.opt_vq, self.opt_disc = self.configure_optimizers(args)
        self.resumed_epoch = 0
        if args.checkpoint_path is not None:
            self.vqgan.load_state_dict(torch.load(args.checkpoint_path))
            resumed_epoch = int(args.checkpoint_path.split('_')[-1].split('.')[0])
            self.resumed_epoch = resumed_epoch
            print(f"Resuming training from checkpoint: {args.checkpoint_path}")
            print(f"Resuming from epoch {resumed_epoch}")

        if args.checkpoint_vq_opt_path is not None and args.checkpoint_disc_opt_path is not None:
            self.opt_vq.load_state_dict(torch.load(args.checkpoint_vq_opt_path))
            self.opt_disc.load_state_dict(torch.load(args.checkpoint_disc_opt_path))
            
        self.prepare_training()

        self.train(args)

    def load_data_2(self, folder, batch_size):
        transform = Compose([Resize((args.image_size, args.image_size)), ToTensor()])
        dataset = CustomImageDataset(folder, transform=transform)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
        

    def configure_optimizers(self, args):
        lr = args.learning_rate
        # vq_vae part optimizer
        opt_vq = torch.optim.Adam(
            list(self.vqgan.encoder.parameters()) +
            list(self.vqgan.decoder.parameters()) +
            list(self.vqgan.codebook.parameters()) +
            list(self.vqgan.quant_conv.parameters()) +
            list(self.vqgan.post_quant_conv.parameters()),
            lr=lr, eps=1e-08, betas=(args.beta1, args.beta2)
        )
        # discriminator part. 
        opt_disc = torch.optim.Adam(self.discriminator.parameters(),
                                    lr=lr, eps=1e-08, betas=(args.beta1, args.beta2))

        return opt_vq, opt_disc

    @staticmethod
    def prepare_training():
        os.makedirs("results_2700Images_with_pos_pruned_v1", exist_ok=True)
        os.makedirs("checkpoints", exist_ok=True)
        folder_name = args.train_folder
        # os.makedirs(os.path.join("results", folder_name), exist_ok=True)

    def train(self, args):
        # train_dataset = load_data(args)
        # steps_per_epoch = len(train_dataset)
        tensorboard_log_dir = f'runs/lr={args.learning_rate}_with_pos_2700Imagespruned_v1_totalepochs={args.epochs}_perceptual_loss_factor={args.perceptual_loss_factor}_discfactor={args.disc_start}_codebook={args.num_codebook_vectors}_latentdim={args.latent_dim}_{time.strftime("%Y%m%d-%H%M%S")}'
        writer = SummaryWriter(log_dir=tensorboard_log_dir)
        steps_per_epoch = len(self.train_loader)
        # output_path = os.path.join(args.train_folder, "results")
        
        
        for epoch in range(self.resumed_epoch+1,args.epochs):
            total_vq_loss = 0  # initialize total_vq_loss
            total_gan_loss = 0  # initialize total_gan_loss
            # The tqdm function is being used here to provide a progress bar for each epoch of training. 
            with tqdm(range(len(self.train_loader))) as pbar:
                
                # In this particular case, the zip function is used to iterate 
                # over both the progress bar and the training dataset simultaneously.
                for i, imgs in enumerate(self.train_loader):
                    # imgs.shape torch.Size([5, 3, 256, 256]) ==> first dimension is batch size(5)
                    # remaining dimension is 3,256,256
                    imgs = imgs.to(device=args.device)
                    decoded_images, _, q_loss = self.vqgan(imgs)

                    disc_real = self.discriminator(imgs)
                    disc_fake = self.discriminator(decoded_images)

                    disc_factor = self.vqgan.adopt_weight(args.disc_factor, epoch*steps_per_epoch+i, threshold=args.disc_start)
                    
                    print("imgs.shape",imgs.shape)
                    # decoded_images, _, q_loss = self.vqgan(imgs)
                    print("decoded_images.shape",decoded_images.shape)
                
                    perceptual_loss = self.perceptual_loss(imgs, decoded_images)
                    rec_loss = torch.abs(imgs - decoded_images)
                    perceptual_rec_loss = args.perceptual_loss_factor * perceptual_loss + args.rec_loss_factor * rec_loss
                    perceptual_rec_loss = perceptual_rec_loss.mean()
                    g_loss = -torch.mean(disc_fake)

                    λ = self.vqgan.calculate_lambda(perceptual_rec_loss, g_loss)
                    vq_loss = perceptual_rec_loss + q_loss + disc_factor * λ * g_loss

                    d_loss_real = torch.mean(F.relu(1. - disc_real))
                    d_loss_fake = torch.mean(F.relu(1. + disc_fake))
                    gan_loss = disc_factor * 0.5*(d_loss_real + d_loss_fake)

                    # Backward loss.. 
                    
                    self.opt_vq.zero_grad()
                    vq_loss.backward(retain_graph=True)

                    self.opt_disc.zero_grad()
                    gan_loss.backward()

                    self.opt_vq.step()
                    self.opt_disc.step()

                    # if i % 10 == 0:
                    #     with torch.no_grad():
                    #         real_fake_images = torch.cat((imgs[:4], decoded_images.add(1).mul(0.5)[:4]))
                    #         vutils.save_image(real_fake_images, os.path.join("results", f"{epoch}_{i}.jpg"), nrow=4)
                    total_vq_loss += vq_loss.item()
                    total_gan_loss += gan_loss.item()
                    
                    pbar.set_postfix(
                        VQ_Loss=np.round(vq_loss.cpu().detach().numpy().item(), 5),
                        GAN_Loss=np.round(gan_loss.cpu().detach().numpy().item(), 3)
                    )
                    pbar.update(0)
                
                # assuming real_fake_images, epoch, i are defined
                # and args is an instance of Args class with train_folder attribute defined
                
                
                with torch.no_grad():
                    real_fake_images = torch.cat((imgs[:4], decoded_images.add(1).mul(0.5)[:4]))
                    vutils.save_image(real_fake_images, os.path.join("results_2700Images_with_pos_pruned_v1", f"{epoch}_{i}_{args.latent_dim}_{args.num_codebook_vectors}_{args.disc_start}_{args.epochs}.jpg"), nrow=4)
                    
                avg_vq_loss_validation, avg_gan_loss_validation = self.validate(epoch)
                # torch.save(self.vqgan.state_dict(), os.path.join("checkpoints", f"vqgan_epoch_{epoch}.pt"))
                
                avg_vq_loss = total_vq_loss / len(self.train_loader)
                avg_gan_loss = total_gan_loss / len(self.train_loader)
                print(f'Training losses after epoch {epoch}: VQ Loss={avg_vq_loss}, GAN Loss={avg_gan_loss}')

                
                # print(f'Training losses after epoch {epoch}: VQ Loss={avg_vq_loss}, GAN Loss={avg_gan_loss}')
                # # Saving the optimizer state
                # torch.save(self.opt_vq.state_dict(), os.path.join("checkpoints", f"opt_vq_epoch_{epoch}.pt"))
                # torch.save(self.opt_disc.state_dict(), os.path.join("checkpoints", f"opt_disc_epoch_{epoch}.pt"))
                # Log losses and metrics for each training iteration:
            writer.add_scalar('Training VQ Loss', vq_loss.item(), epoch)
            writer.add_scalar('Training GAN Loss', gan_loss.item(), epoch)

            #  Log average losses and metrics for each epoch:
            writer.add_scalar('Validation VQ Loss', avg_vq_loss_validation, epoch)
            writer.add_scalar('Validation GAN Loss', avg_gan_loss_validation, epoch)

        writer.close()

    def validate(self, epoch):
        with torch.enable_grad():
            total_vq_loss = 0
            total_gan_loss = 0
            # for i, (imgs, _) in enumerate(self.valid_loader):
            for i, imgs in enumerate(self.valid_loader):
                imgs = imgs.to(device=args.device)
                print("imgs.shape",imgs.shape)
                decoded_images, _, q_loss = self.vqgan(imgs)
                print("decoded_images.shape",decoded_images.shape)
                disc_real = self.discriminator(imgs)
                disc_fake = self.discriminator(decoded_images)
                disc_factor = self.vqgan.adopt_weight(args.disc_factor, epoch*len(self.valid_loader)+i, threshold=args.disc_start)

                perceptual_loss = self.perceptual_loss(imgs, decoded_images)
                rec_loss = torch.abs(imgs - decoded_images)
                perceptual_rec_loss = args.perceptual_loss_factor * perceptual_loss + args.rec_loss_factor * rec_loss
                perceptual_rec_loss = perceptual_rec_loss.mean()
                g_loss = -torch.mean(disc_fake)

                # with torch.enable_grad():  # enabling gradient computation temporarily
                λ = self.vqgan.calculate_lambda(perceptual_rec_loss, g_loss)
                vq_loss = perceptual_rec_loss + q_loss + disc_factor * λ * g_loss
                # total_vq_loss += vq_loss.item()

                d_loss_real = torch.mean(F.relu(1. - disc_real))
                d_loss_fake = torch.mean(F.relu(1. + disc_fake))
                gan_loss = disc_factor * 0.5*(d_loss_real + d_loss_fake)
                total_gan_loss += gan_loss.item()
                total_vq_loss += vq_loss.item()
                # total_gan_loss += gan_loss.item()
                

            avg_vq_loss_validation = total_vq_loss / len(self.valid_loader)
            avg_gan_loss_validation = total_gan_loss / len(self.valid_loader)
            print(f'Validation losses after epoch {epoch}: VQ Loss={avg_vq_loss_validation}, GAN Loss={avg_gan_loss_validation}')

        return avg_vq_loss_validation, avg_gan_loss_validation    # # Log average losses and metrics for each epoch:
    #     writer.add_scalar('Validation VQ Loss', avg_vq_loss, epoch)
    #     writer.add_scalar('Validation GAN Loss', avg_gan_loss, epoch)    

  





# Just to show how you can use the arguments:
# train_vqgan = TrainVQGAN(args)

class Args:
    def __init__(self,train_folder_name,valid_folder_name, latent_dim,codebook,disc_value):
        
        
        self.latent_dim = latent_dim
        self.image_size = 256
        self.num_codebook_vectors = codebook
        self.beta = 0.25
        self.image_channels = 3
        self.dataset_path = '/data'
        self.device = 'cuda'
        self.batch_size = 5
        self.epochs = 1000
        self.learning_rate = 2.25e-05
        self.beta1 = 0.5
        self.beta2 = 0.9
        self.disc_start = disc_value
        self.disc_factor = 1.
        self.rec_loss_factor = 1.
        self.perceptual_loss_factor = 1
        self.checkpoint_path = None
        self.checkpoint_vq_opt_path = None
        self.checkpoint_disc_opt_path = None
        self.train_folder = os.path.join(os.getcwd(), train_folder_name)
        self.valid_folder = os.path.join(os.getcwd(), valid_folder_name)
        # Create a unique name for the tensorboard log directory
        

        # Define the folder names
# train_folder_name = ['train_all_only_few','train_all', 'train_all_8K']
# valid_folder_name = ['valid_all_only_few','valid_all', 'valid_all_8K']

train_folder_name = ['train_all', 'train_all_8K']
valid_folder_name = ['valid_all', 'valid_all_8K']

codebook_vector = [8192]
latent_dims = [256]
disc_starts = [100]
# perceptual_loss_factors = [0.5]
# rec_loss_factors = [0.5]
# for i in range(1,len(train_folder_name)):
for i in range(1):
    
    for latent_dim in latent_dims:
        for codebook in codebook_vector:
            for disc_value in disc_starts:
                args = Args(train_folder_name[i], valid_folder_name[i], latent_dim,codebook, disc_value)



                # Just to show how you can use the arguments:
                train_vqgan = TrainVQGAN(args)



            # if __name__ == '__main__':
            #     parser = argparse.ArgumentParser(description="VQGAN")
            #     parser.add_argument('--latent-dim', type=int, default=256, help='Latent dimension n_z (default: 256)')
            #     parser.add_argument('--image-size', type=int, default=256, help='Image height and width (default: 256)')
            #     parser.add_argument('--num-codebook-vectors', type=int, default=1024, help='Number of codebook vectors (default: 256)')
            #     parser.add_argument('--beta', type=float, default=0.25, help='Commitment loss scalar (default: 0.25)')
            #     parser.add_argument('--image-channels', type=int, default=3, help='Number of channels of images (default: 3)')
            #     parser.add_argument('--dataset-path', type=str, default='/data', help='Path to data (default: /data)')
            #     parser.add_argument('--device', type=str, default="cuda", help='Which device the training is on')
            #     parser.add_argument('--batch-size', type=int, default=5, help='Input batch size for training (default: 6)')
            #     parser.add_argument('--epochs', type=int, default=2, help='Number of epochs to train (default: 50)')
            #     parser.add_argument('--learning-rate', type=float, default=2.25e-05, help='Learning rate (default: 0.0002)')
            #     parser.add_argument('--beta1', type=float, default=0.5, help='Adam beta param (default: 0.0)')
            #     parser.add_argument('--beta2', type=float, default=0.9, help='Adam beta param (default: 0.999)')
            #     parser.add_argument('--disc-start', type=int, default=100, help='When to start the discriminator (default: 0)')
            #     parser.add_argument('--disc-factor', type=float, default=1., help='')
            #     parser.add_argument('--rec-loss-factor', type=float, default=1., help='Weighting factor for reconstruction loss.')
            #     parser.add_argument('--perceptual-loss-factor', type=float, default=1., help='Weighting factor for perceptual loss.')

            #     parser.add_argument('--checkpoint-path', type=str, default=None, help='Path to the checkpoint to resume training from (default: None)')
            #     parser.add_argument('--checkpoint-vq-opt-path', type=str, default=None, help='Path to the VQ optimizer checkpoint to resume training from (default: None)')
            #     parser.add_argument('--checkpoint-disc-opt-path', type=str, default=None, help='Path to the discriminator optimizer checkpoint to resume training from (default: None)')
            #     #  add some new arguments
            #     # parser.add_argument('--train-folder', type=str, default='/VQGANs/train_all/', help='Path to training data folder')
            #     # parser.add_argument('--valid-folder', type=str, default='/VQGANs/valid_all/', help='Path to validation data folder')
            # #
            #     # parser.add_argument('--train-folder', type=str, default='/home/verma.lu/VQGANs/train_all/', help='Path to training data folder')
            #     # parser.add_argument('--valid-folder', type=str, default='/home/verma.lu/VQGANs/valid_all/', help='Path to validation data folder')

            #     parser.add_argument('--train-folder', type=str, default='./train_all/', help='Path to training data folder')
            #     parser.add_argument('--valid-folder', type=str, default='./valid_all/', help='Path to validation data folder')
            #     args = parser.parse_args()
            #     args.checkpoint_path = None
            #     args.checkpoint_disc_opt_path = None
            #     args.checkpoint_vq_opt_path = None

            #     # args.dataset_path = r"C:\Users\luvve\flowersSix\jpg"


            #     # args.dataset_path = r"C:\Users\luvve\flowers\jpg"
            #     # args.checkpoint_path = r"C:\Users\luvve\VQGAN\checkpoints\vqgan_epoch_61.pt"
            #     # args.checkpoint_vq_opt_path = r"C:\Users\luvve\VQGAN\checkpoints\opt_vq_epoch_61.pt"
            #     # args.checkpoint_disc_opt_path = r"C:\Users\luvve\VQGAN\checkpoints\opt_disc_epoch_61.pt"

            #     train_vqgan = TrainVQGAN(args)





/home/verma.lu/.conda/envs/pytorch_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/verma.lu/.conda/envs/pytorch_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/297 [00:00<?, ?it/s]

imgs.shape torch.Size([5, 3, 256, 256])
decoded_images.shape torch.Size([5, 3, 256, 256])


  0%|          | 0/297 [00:04<?, ?it/s, GAN_Loss=1.14, VQ_Loss=2.13]

imgs.shape torch.Size([5, 3, 256, 256])
decoded_images.shape torch.Size([5, 3, 256, 256])


  0%|          | 0/297 [00:05<?, ?it/s, GAN_Loss=1.1, VQ_Loss=1.89] 

...

  0%|          | 0/297 [01:22<?, ?it/s, GAN_Loss=0, VQ_Loss=0.0724]    

imgs.shape torch.Size([5, 3, 256, 256])
decoded_images.shape torch.Size([5, 3, 256, 256])


  0%|          | 0/297 [01:23<?, ?it/s, GAN_Loss=0, VQ_Loss=0.127] 

imgs.shape torch.Size([5, 3, 256, 256])
decoded_images.shape torch.Size([5, 3, 256, 256])


  0%|          | 0/297 [01:23<?, ?it/s, GAN_Loss=0.023, VQ_Loss=0.0726]

In [ ]:
import shutil
shutil.make_archive('results', 'zip', 'results')

In [2]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/.ipynb_checkpoints/ (stored 0%)
  adding: results/.ipynb_checkpoints/131_296_256_2048-checkpoint.jpg (deflated 1%)
  adding: results/.ipynb_checkpoints/1_296_256_2048-checkpoint.jpg (deflated 7%)
  adding: results/.ipynb_checkpoints/1_296_256_512-checkpoint.jpg (deflated 5%)
  adding: results/.ipynb_checkpoints/23_296_256_512-checkpoint.jpg (deflated 1%)
  adding: results/.ipynb_checkpoints/298_296_256_2048-checkpoint.jpg (deflated 1%)
  adding: results/.ipynb_checkpoints/298_296_256_512-checkpoint.jpg (deflated 1%)
  adding: results/.ipynb_checkpoints/299_296_256_2048-checkpoint.jpg (deflated 2%)
  adding: results/.ipynb_checkpoints/299_296_256_512-checkpoint.jpg (deflated 1%)
  adding: results/100_296_256_2048.jpg (deflated 1%)
  adding: results/100_296_256_512.jpg (deflated 1%)
  adding: results/101_296_256_2048.jpg (deflated 1%)
  adding: results/101_296_256_512.jpg (deflated 1%)
  adding: results/102_296_256_2048.jpg (deflated 1%)
 

In [17]:
# !pip install albumentations

# !pip install matplotlib

In [23]:
import os

current_path = os.getcwd()
print(current_path)

/home/verma.lu/VQGANs


In [19]:
%tb

SystemExit: 2